In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


See the steps uptil where we conver the categorical data into numerical stays same with the difference that we will be applying one-hot encoding instead of using 'replace' function. 

The plan is also to figure out outliers in the dataset and get rid of them. Extracting infromation from the data could be done by visualization, but i don't think that will help in achieving our goal. However we are not neglecting the fact that there might be some correlation between the variables that's why th  we will be performing correlation check. 

Moving ahead with it, i guess we should now focues on providing rich data to the model and try every means.

### Converting into numerical data

In [4]:
numerical_values = [cn for cn in df.columns if df[cn].dtype!='object']

In [5]:
df['Age'] = df['Age']/365.25
df['Age'] = df['Age'].round(0) 

In [6]:
categorical_values = [cn for cn in df.columns if df[cn].dtype=='object']

In [7]:
categorical_values = categorical_values[:-1]

In [8]:
categorical_values

['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

In [9]:
from sklearn.preprocessing import OneHotEncoder
# Subset your DataFrame with only the categorical columns
df_categorical = df[categorical_values]

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False, drop='first')  # Setting drop='first' to drop one of the binary columns per category

# Fit and transform the categorical columns
df_encoded = pd.DataFrame(encoder.fit_transform(df_categorical), columns=encoder.get_feature_names_out(categorical_values))

# Drop the original categorical columns from the DataFrame
df = df.drop(columns=categorical_values)

# Concatenate the one-hot encoded columns with the original DataFrame
df = pd.concat([df, df_encoded], axis=1)

C:\Users\Aman Shrivastav\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
df.head()

,id,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,...,Prothrombin,Stage,Status,Drug_Placebo,Sex_M,Ascites_Y,Hepatomegaly_Y,Spiders_Y,Edema_S,Edema_Y
0,0,999,59.0,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,...,9.7,3.0,D,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,2574,53.0,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,...,11.0,3.0,C,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3428,38.0,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,...,11.7,4.0,D,1.0,0.0,0.0,1.0,1.0,0.0,1.0
3,3,2576,51.0,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,...,10.7,3.0,C,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,788,46.0,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,...,10.6,4.0,C,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7905 entries, 0 to 7904
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              7905 non-null   int64  
 1   N_Days          7905 non-null   int64  
 2   Age             7905 non-null   float64
 3   Bilirubin       7905 non-null   float64
 4   Cholesterol     7905 non-null   float64
 5   Albumin         7905 non-null   float64
 6   Copper          7905 non-null   float64
 7   Alk_Phos        7905 non-null   float64
 8   SGOT            7905 non-null   float64
 9   Tryglicerides   7905 non-null   float64
 10  Platelets       7905 non-null   float64
 11  Prothrombin     7905 non-null   float64
 12  Stage           7905 non-null   float64
 13  Status          7905 non-null   object 
 14  Drug_Placebo    7905 non-null   float64
 15  Sex_M           7905 non-null   float64
 16  Ascites_Y       7905 non-null   float64
 17  Hepatomegaly_Y  7905 non-null   f

In [12]:
numerical_values = numerical_values[1:-1]

In [13]:
numerical_values

['N_Days',
 'Age',
 'Bilirubin',
 'Cholesterol',
 'Albumin',
 'Copper',
 'Alk_Phos',
 'SGOT',
 'Tryglicerides',
 'Platelets',
 'Prothrombin']

In [14]:
df[numerical_values].head()

,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin
0,999,59.0,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7
1,2574,53.0,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0
2,3428,38.0,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7
3,2576,51.0,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7
4,788,46.0,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6


In [15]:
main_df = df

In [16]:
main_df.head()

,id,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,...,Prothrombin,Stage,Status,Drug_Placebo,Sex_M,Ascites_Y,Hepatomegaly_Y,Spiders_Y,Edema_S,Edema_Y
0,0,999,59.0,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,...,9.7,3.0,D,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,2574,53.0,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,...,11.0,3.0,C,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3428,38.0,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,...,11.7,4.0,D,1.0,0.0,0.0,1.0,1.0,0.0,1.0
3,3,2576,51.0,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,...,10.7,3.0,C,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,788,46.0,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,...,10.6,4.0,C,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [17]:
from imblearn.over_sampling import RandomOverSampler

X = main_df.drop('Status', axis=1) 
y = main_df['Status']  

oversampler = RandomOverSampler(random_state=42)

X_resampled, y_resampled = oversampler.fit_resample(X, y)

df_resampled = pd.concat([X_resampled, y_resampled], axis=1)

print(df_resampled['Status'].value_counts())


Status
D     4965
C     4965
CL    4965
Name: count, dtype: int64


In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X = df_resampled.drop('Status',axis=1)
y = df_resampled['Status']

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [21]:
from sklearn.ensemble import RandomForestClassifier

rf= RandomForestClassifier()

# Train the model on the resampled training data
rf.fit(X_train, y_train)

RandomForestClassifier()

In [22]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = rf.predict(X_test)

# Evaluate the model's performance on the original test set
accuracy = accuracy_score(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)

print("Accuracy on Resampled Data:", accuracy)
print("Classification Report on Resampled Data:\n", classification_report)

Accuracy on Resampled Data: 0.9425981873111783
Classification Report on Resampled Data:
               precision    recall  f1-score   support

           C       0.94      0.90      0.92      1045
          CL       1.00      1.00      1.00      1013
           D       0.89      0.93      0.91       921

    accuracy                           0.94      2979
   macro avg       0.94      0.94      0.94      2979
weighted avg       0.94      0.94      0.94      2979



In [23]:
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Assuming X and y are your feature matrix and target variable
# Convert string labels to numerical labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Create XGBoost model
model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(set(y_encoded)))

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Decode the numerical predictions back to original labels
y_pred_original = label_encoder.inverse_transform(y_pred)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_rep)


Accuracy: 0.9308492782813025

Confusion Matrix:
[[ 920    9  116]
 [   0 1013    0]
 [  75    6  840]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.88      0.90      1045
           1       0.99      1.00      0.99      1013
           2       0.88      0.91      0.90       921

    accuracy                           0.93      2979
   macro avg       0.93      0.93      0.93      2979
weighted avg       0.93      0.93      0.93      2979



In [24]:
testing_data = pd.read_csv('test.csv')

In [25]:
testing_data['Age'] = testing_data['Age']/365.25
testing_data['Age'] = testing_data['Age'].round(0) 

In [26]:
categorical_values = [cn for cn in testing_data.columns if testing_data[cn].dtype=='object']

In [27]:
categorical_values

['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

In [28]:
columns_to_replace = categorical_values

In [29]:
columns_to_replace

['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

In [30]:
from sklearn.preprocessing import OneHotEncoder
# Subset your DataFrame with only the categorical columns
df_categorical = testing_data[categorical_values]

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False, drop='first')  # Setting drop='first' to drop one of the binary columns per category

# Fit and transform the categorical columns
df_encoded = pd.DataFrame(encoder.fit_transform(df_categorical), columns=encoder.get_feature_names_out(categorical_values))

# Drop the original categorical columns from the DataFrame
df = testing_data.drop(columns=categorical_values)

# Concatenate the one-hot encoded columns with the original DataFrame
df = pd.concat([df, df_encoded], axis=1)

C:\Users\Aman Shrivastav\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [57]:
probabilities = model.predict_proba(df)

# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'id': df['id'],
    'Status_C': probabilities[:, 0],
    'Status_CL': probabilities[:, 1],
    'Status_D': probabilities[:, 2]
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file3.csv', index=False)

In [60]:
#Submission file 3 got me a score of 0.48 so from 0.52 i was able to bring it down to 0.48(XGB)